In [16]:
import pandas as pd
import pickle
import torch

from sklearn.preprocessing import MinMaxScaler

In [17]:
def load_min_max_scaler()-> MinMaxScaler:
    with open('c:/workspace/ae_min_max_scaler.skl', 'rb') as f:
        std_scaler = pickle.load(f)

    return std_scaler

class AutoEncoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(241, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 18),
            torch.nn.ReLU(),
            torch.nn.Linear(18, 9)
        )

        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(9, 18),
            torch.nn.ReLU(),
            torch.nn.Linear(18, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 241),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)

        return decoded

In [18]:
# Load the dataset
df = pd.read_csv('c:/workspace/dillon.csv')

# Remove data with NaN values
df = df.dropna()

# Separate the datasets
X = df.drop(['date', 'hour'], axis=1)

# Scale the X dataset
ss = load_min_max_scaler()
X = ss.transform(X)

# Convert to tensor
X = torch.from_numpy(X).float()

In [19]:
model = AutoEncoder()
model.load_state_dict(torch.load('c:/workspace/ae_fnn.pt'))
model.eval()

AutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=241, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=36, bias=True)
    (3): ReLU()
    (4): Linear(in_features=36, out_features=18, bias=True)
    (5): ReLU()
    (6): Linear(in_features=18, out_features=9, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=9, out_features=18, bias=True)
    (1): ReLU()
    (2): Linear(in_features=18, out_features=36, bias=True)
    (3): ReLU()
    (4): Linear(in_features=36, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=241, bias=True)
    (7): Sigmoid()
  )
)

In [20]:
df = df.reset_index()
output = model(X).detach().numpy()
output = ss.inverse_transform(output)
output = pd.DataFrame(output, columns=df.columns[3:])
output.insert(0, 'date', df['date'] )
output.insert(1, 'hour', df['hour'] )
output.head()

,date,hour,daynum,0_p,0_t,0_rh,0_dp,0_u,0_v,1_p,...,38_rh,38_dp,38_u,38_v,39_p,39_t,39_rh,39_dp,39_u,39_v
0,01/01/2010,0,-0.843587,98793.726562,-105.679504,822.686951,-129.939163,1.308561,-4.391144,96555.703125,...,145.432571,-674.531616,292.306793,-94.848557,12219.458008,-533.684204,140.248245,-690.185303,266.477081,-73.886414
1,01/04/2010,0,-0.774481,98933.296875,-109.025673,843.185486,-134.981659,-7.800784,-10.032429,96707.109375,...,127.630379,-674.830139,176.778366,-77.173950,12223.125000,-523.351501,118.513824,-691.125427,169.102081,-59.261620
2,01/05/2010,0,-0.730303,98974.335938,-110.082222,880.879639,-121.073799,1.068276,-16.417421,96738.093750,...,96.150696,-697.289124,175.549728,-66.871056,12224.039062,-518.714661,88.504219,-711.572815,168.342422,-48.810001
3,01/05/2010,12,-0.829424,99433.179688,-131.616913,748.807495,-168.625870,-6.172839,0.556936,97204.812500,...,224.024658,-674.775879,228.097137,-139.773911,12236.047852,-549.626587,202.263000,-692.973511,217.582733,-116.887917
4,01/06/2010,12,-0.658801,97936.117188,-27.551332,838.019165,-55.333946,-3.052285,-7.353895,95722.164062,...,122.746590,-683.396912,284.569489,-25.472088,12198.258789,-538.841309,113.814926,-701.413391,251.452667,-18.668728


In [21]:
df.head()

,index,date,hour,daynum,0_p,0_t,0_rh,0_dp,0_u,0_v,...,38_rh,38_dp,38_u,38_v,39_p,39_t,39_rh,39_dp,39_u,39_v
0,0,01/01/2010,0,-1.00,98671.0,-112.0,770.0,-144.0,14.1,-15.6,...,166.1,-678.6,298.1,-21.4,12216.8,-547.5,161.0,-689.6,307.5,18.6
1,6,01/04/2010,0,-1.00,99106.0,-143.0,750.0,-178.0,-1.6,-14.9,...,142.7,-686.0,122.7,-56.4,12227.6,-531.7,107.4,-705.2,129.0,-76.8
2,8,01/05/2010,0,-1.00,98967.0,-144.0,800.0,-171.0,9.7,-2.2,...,121.5,-713.8,160.8,-156.0,12224.2,-560.6,113.4,-726.2,140.0,-135.3
3,9,01/05/2010,12,-1.00,98925.0,-259.0,780.0,-286.0,10.6,-10.6,...,259.0,-677.1,157.4,-152.1,12223.1,-558.4,252.4,-666.1,174.8,-146.2
4,11,01/06/2010,12,-0.99,97939.0,-123.0,770.0,-155.0,-32.1,39.6,...,182.2,-711.7,237.6,-115.6,12198.5,-561.4,126.9,-719.0,201.7,-103.5
